# Why Profiling is needed?

Profiling is important to understand which parts of program consume most of the resources. This helps to improve the slow executing code segments to perform better or to focus on finding alternatives. Not only CPU, but also memory(RAM), disk usage(Disk I/O), network operations etc. also can be measured to determine bottlenecks of a program.

The basic method of identifying the bottlenecks is understanding the time consumption of the program sections. In jupyter notebook we can use `%%timeit` magic, time.time() or time decorators. In order to test the mentioned techniques we will define a special function named `Julia Set` which is Heavy CPU bound and less memory consuming non linear time consuming task. More technically speaking this is a fractal function which generates a complex output image.


<center><image src="./img/1.jpg" width="200px" /></center>

The basic psuedo code for calculation is as follows. In here coordinates are imaginary numbers and max_iter is a predefined variable for the function.
<pre style='color:yellow'> 
coordinates = []

for z in coordinates:
    for _ in range(max_iter):
        
        if (abs(z)< thres):
            z = z*z + c
        else:
            break
</pre>

But for the sake of testing various scenarios following imlpementation has few other parts added to it.

In [1]:
import time
import cv2

# area of imaginary space to calculate pixel values
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_img = -0.62772, -.42193

In [7]:
def display_img(arr):
    '''
    Function to display the generated output as an image.
    '''
    import numpy as np
    arr = np.array(arr)
    arr = arr.reshape((int(len(arr)**0.5), int(len(arr)**0.5)), order='C')

    arr = np.array(arr, dtype=np.uint8)
    cv2.imshow("Julia set", arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def calculate_juliaset_serial(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

In [9]:
def calc_juliaset_time(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs), to build Julia set"""

    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width

    x = []
    y = []

    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step

    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_img))
    
    print("Length of x:", len(x))
    print("Total elements:", len(zs))

    start_time = time.time()
    output = calculate_juliaset_serial(max_iterations, zs, cs)
    end_time = time.time()

    secs = end_time - start_time
    print(calculate_juliaset_serial.__name__ + " took", secs, "seconds")

    return output

In [10]:
# reasonable defaults for a laptop
val = calc_juliaset_time(desired_width=1000, max_iterations=300)
display_img(val)

Length of x: 1000
Total elements: 1000000
calculate_juliaset_serial took 6.378915786743164 seconds


As above we can use the julia set to as a baseline task to check the performance. In the above case we have used the good old print statement with time difference to measure the performance. But this time change with the other processes running in the computer. Also print statements like above causes inconvienience in the long run. Instead we can use a decorator to measure time and print. (Or in Jupyter notebooks magic functions :D )

In [11]:
from functools import wraps

def timefn(fn):

    @wraps(fn)
    def measure_time(*args, **kwargs):

        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()

        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    
    return measure_time


In [12]:
@timefn
def calc_juliaset_time(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs), to build Julia set"""

    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width

    x = []
    y = []

    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step

    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_img))
    
    print("Length of x:", len(x))
    print("Total elements:", len(zs))

    output = calculate_juliaset_serial(max_iterations, zs, cs)

    return output

In [13]:
x = calc_juliaset_time(1000, 300)

Length of x: 1000
Total elements: 1000000
@timefn: calc_juliaset_time took 6.546191453933716 seconds


Other than above methods, we can use python provided `timeit` function. It provides more functionality to time a execution with repetitions and loops.

Also it is important to keep track of other computer processes, because sometimes those may cause sudden spikes in CPU usage which may affect our profiling process.

Outside of python, we can use OS provided functionalities such as UNIX `time` command to measure program execution time as well.

<center>

__time --verbose python_script_name.py__

</center>

Another way to do the profiling is using the `cProfile` or `Profile` modules provided in the standard library.

<center>Eg:-

__python -m cProfile -s cumulative python_script_name.py__

</center>

Below is a example execution of cProfile on juliaset calculation script. As we can see it measures the execution times, number of calls to each function with little bit of overhead to the cProfiler itself. This is a comparatively descriptive way of analyzing our code to identify bottlenecks.

<center><image src="./img/2.jpg" width="500px" /></center>

A point to note is that cProfile gives details on funcion call basis, not line basis. So it would be bit harder to pinpoint problematic location of the code.

Also we can write the cProfile output to a statistics file which can later be read by python itself. This way we can further analyze the details regarding our program.

<center>Eg:-

__python -m cProfile -o profile.stats python_script_name.py__

</center>

Above command will write a file named profile.stats and we can use it as below in python.

In [14]:
import pstats

p = pstats.Stats('scripts/profile.stats')
p.sort_stats('cumulative')
p.print_stats()

Fri Apr 29 12:13:13 2022    scripts/profile.stats

         36221991 function calls in 10.417 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.417   10.417 {built-in method builtins.exec}
        1    0.020    0.020   10.417   10.417 juliset.py:2(<module>)
        1    0.405    0.405   10.396   10.396 juliset.py:18(calc_juliaset_time)
        1    6.102    6.102    9.842    9.842 juliset.py:5(calculate_juliaset_serial)
 34219980    3.740    0.000    3.740    0.000 {built-in method builtins.abs}
  2002000    0.149    0.000    0.149    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [15]:
p.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
{built-in method builtins.exec}                   <- 
juliset.py:2(<module>)                            <-       1    0.020   10.417  {built-in method builtins.exec}
juliset.py:18(calc_juliaset_time)                 <-       1    0.405   10.396  juliset.py:2(<module>)
juliset.py:5(calculate_juliaset_serial)           <-       1    6.102    9.842  juliset.py:18(calc_juliaset_time)
{built-in method builtins.abs}                    <- 34219980    3.740    3.740  juliset.py:5(calculate_juliaset_serial)
{method 'append' of 'list' objects}               <- 2002000    0.149    0.149  juliset.py:18(calc_juliaset_time)
{built-in method builtins.print}                  <-       2    0.000    0.000  juliset.py:18(calc_juliaset_time)
{built-in method builtins.len}                    <-       2    0.000    0.000  juliset.py

# Line Profiler

One of the other strong tool to identify CPU bound problems is `Line-Profiler`. (pip install line_profiler). In using this tool, following values are used.

- @profile decorator is used to mark the function need to profile.
- kernprof script is used to execute the code.
- '-l' for line by line profiling
- '-v' for verbose output, if we dont use this argument we will get  an .lprof file as an output which we can analyze later.

<center>

**kernprof -l -v python_script_name.py**
</center>


<center><image src="./img/3.jpg" width="700px" /></center>

Points to note is that, this profile takes substantial amount of time to run. 

Also this reflects interesting insights related to the code we are checking. For example in the above code the line that takes the most amount of time is the while clause, rather than the multiplication part. Even __n ++1__ takes 27% of the total time. (This is apparantly due to the python language dynamic type feature. We will check later.) As for the while loop in order to identify which parts takes the most time lets break it down and analyze again.

<center><image src="./img/4.jpg" width="700px" /></center>

Assume the below hypothesis.

`Since python statements execute from left to right and opportunistic, it can be helpful to improve performance if we put chepest test(execution wise) to the left.`

If we test this hypothesis we will see that, program does not provide any significant evidence to support that. Which means order of logic test in a statement does not guarantee any reliable performance improvement.

# Memory Profiler

Just as we can use line profiler to check the execution bottlenecks, we can use memory profilers to check the memory usage of out programs to improve the performance. To do that we can use the python package `memory_profiler`.

<center>

**pip install memory_profiler**

**pip install psutil (to improve profiling speed)**
</center>

It should be noted that memory profiling is significantly slower than the all previously mentioned techniques. Therefore, it makes sense to run the profiling on a small code part.

To use the memory profiler use below command. This outputs line by line statistics of memory usage of the script we are analyzing.
<center>

**python -m memory_profiler script_name.py**
</center>

Also we can use below command to get memory use as samples over time. The output of that command can be used to visualize the memory usage as a plot.

<center>

**mprof run python script.py**

To use the output --> **mprof plot**

To use the memprofiler in Jupyter notebooks first run **%load_ext memory_profiler**
</center>
</br>

<center><image src="./img/5.png" width="700px"/></center>

This graph is not accurate due to a problem of my machine at the time!

</br>
To add more sense to the above plot, we can use labels inside of the code like below. Check the context manager part(with clause part).

<pre style='color:yellow'>
@profile
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""

    with profile.timestamp("create_output_list"):
        output = [0] * len(zs)

    time.sleep(1)

    with profile.timestamp("calculate_output"):   
        for i in range(len(zs)):
            n = 0
            z = zs[i]
            c = cs[i]
            while n < maxiter and abs(z) < 2:
                z = z * z + c
                n += 1
            output[i] = n
return output
</pre>

If we remove the creation of zs and cs list and then add it to the `calculate_juliaset_serial_expanded` function it self, we can reduce the ram usage by almost half. We can verify this by changing the code and plotting the memory usage again.

<strong>
We can use Ipython %memit function to measure the memory usage as well. (just like %timeit)
</strong>

# PySpy

This is another type of memory profiler that we can use to measure the memory usages of python processes. This uniqueness of this profiler is that, this is a sampling type memory profiler. Which means it checks the already running python process and reports it data back to the user.

<center>

**pip install py-spy**
</center>

To use this we need to first identify the python process id. Then we need to pass this to the pyspy tool to start profiling.

## Examine the Bytecode

The `dis` module helps us to analyze the underline bytecode used by the Cpython virtual machine. It will helps us to identify the coding patterns which would run fast compared to the others.

[Simple Explanation about python VM](https://www.tutorialspoint.com/internal-working-of-python)

In [16]:
import dis
from scripts.lineprofile_julia import *

dis.dis(calc_juliaset_time)

 49           0 LOAD_GLOBAL              0 (x2)
              2 LOAD_GLOBAL              1 (x1)
              4 BINARY_SUBTRACT
              6 LOAD_FAST                0 (desired_width)
              8 BINARY_TRUE_DIVIDE
             10 STORE_FAST               2 (x_step)

 50          12 LOAD_GLOBAL              2 (y1)
             14 LOAD_GLOBAL              3 (y2)
             16 BINARY_SUBTRACT
             18 LOAD_FAST                0 (desired_width)
             20 BINARY_TRUE_DIVIDE
             22 STORE_FAST               3 (y_step)

 52          24 BUILD_LIST               0
             26 STORE_FAST               4 (x)

 53          28 BUILD_LIST               0
             30 STORE_FAST               5 (y)

 55          32 LOAD_GLOBAL              3 (y2)
             34 STORE_FAST               6 (ycoord)

 56     >>   36 LOAD_FAST                6 (ycoord)
             38 LOAD_GLOBAL              2 (y1)
             40 COMPARE_OP               4 (>)
             42 POP_

- Here First column is the corresponding line of code from original file.
- In the second column `>>` indicates a jump point from there to another location of the file.
- Third column is named as Operation address and its related operation name is in the Fourth column.
- Fifth column contains the parameters (id like) needed for that operation.
- Sixth column contains the annotations to identify the parameters mentioned in fifth column with the original code.

Identifying which codes generate more overhead or performance/readability  problems is important in modern day software engineering.

for example below 2 function do the same thing. But their coding styles are vastly different.

In [17]:
def fn_expressive(upper=1_000_000):
    total = 0
    for n in range(upper):
        total += n
    return total

def fn_compressed(upper=1_000_000):
    return sum(range(upper))

assert fn_expressive() == fn_compressed()

In [18]:
%timeit fn_expressive()

34.4 ms ± 478 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit fn_compressed()

23.8 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As we can see the function which use the built in  function is faster compared to the expressive one. The reason can be seen when we check the bytecode.

In [20]:
dis.dis(fn_expressive)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (total)

  3           4 LOAD_GLOBAL              0 (range)
              6 LOAD_FAST                0 (upper)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                12 (to 26)
             14 STORE_FAST               2 (n)

  4          16 LOAD_FAST                1 (total)
             18 LOAD_FAST                2 (n)
             20 INPLACE_ADD
             22 STORE_FAST               1 (total)
             24 JUMP_ABSOLUTE           12

  5     >>   26 LOAD_FAST                1 (total)
             28 RETURN_VALUE


In [22]:
dis.dis(fn_compressed)

  8           0 LOAD_GLOBAL              0 (sum)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_FAST                0 (upper)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 RETURN_VALUE


Obviously first code have 2 variables to maintain, one jump location due to the for loop and its condition which cause the time overhead.